In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute



# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
comp_cluster = 'opt-ml-cluster'
try:
    comp_trget = ComputeTarget(workspace=ws,name=comp_cluster)
except ComputeTargetException:  
    comp_confg = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4, min_nodes=1)
comp_trget = ComputeTarget.create(ws, comp_cluster, comp_confg)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--C' : choice(0.03,0.3,3,10,30),
    '--max_iter' : choice(25,50,75,100)
})

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=comp_target)
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=5,
    policy=policy,
    estimator=src
)
### YOUR CODE HERE ###

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails

hyperdrive_submission = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())

final_model = best_run.register_model(model_name = 'hypermodel',model_path = '/outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
 data_link = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
input_data = TabularDatasetFactory.from_delimited_files(path=data_link)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=final_data,
    label_column_name='y',
    compute_target = comp_trget,
    max_concurrent_iterations = 3,
    enable_early_stopping = True,
    n_cross_validations=7)


In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
class_automl = exp.submit(automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_class_automl, best_fit_model = class_automl.get_output()
print(best_fit_model)

joblib.dump(best_fit_model, 'outputs/best_fit_automl_model.joblib')
best_class_automl.register_model(model_name = 'best_automl.pkl', model_path = '/outputs/')